In [53]:
# Lucas Xu, Highschool Intern @ UAlberta
#Python 3.11.9, WindowsOS 11 
#UI of a heatmap designed to visualize the amount of people per room
#main var/classes: class PannableCanvas, root, button_zoomin, button_zoomout, slider
import tkinter as tk
import tkinter.font as tkFont
from PIL import Image, ImageTk, ImageDraw
from tkinter import filedialog
from tkinter import simpledialog
from tkinter import messagebox
from tkinter import ttk
import math
import fitz
import csv
import json
import pandas as pd
import os
import tempfile
import shutil
import numpy as np
from enum import Enum
from shapely.geometry import Point, Polygon # for point polygon intersection


In [54]:
# generate sample data: a csv with hour 0-24 and number of people

# import random
# SAMPLE_SIZE=5
# pd.DataFrame({
#     'id': range(SAMPLE_SIZE),
#     'people': [[random.randint(0,100) for i in range(24)] for _ in range(SAMPLE_SIZE)]
# }).to_csv('occupancy.csv', index=False)


In [55]:
# window init
root = tk.Tk()
root.title("HeatMap UI")
root.geometry("1600x1000")
root.config(bg='white', pady=100, padx=100)
# root.state('zoomed')
root.state('normal')

max_width = 900
max_height = 800

In [56]:
# file init stuff

#init filename database
filename_database = set()

#temp dir init
temp_dir = tempfile.mkdtemp()

#Have the path where the executable file will be when further implemented

image_path = "test.png"
data_path = "occupancy.csv"

# just the default that the file explorer will open to.
default_search_path = '/Users/stefan/Documents/life/work/network/localization/app/wifiscans/floormaps/Administration'


In [57]:
# make numpy print in json-friendly way
np.set_printoptions(formatter={'all': lambda x: f'{x:.2f}'})

In [58]:
# canvas params
snap_radius = 0.01 # in json coordinates 
# (so 0.01 is 1% of the width/height of the map. Ellipse shaped snap radius I guess)

min_zoom = 0.1
max_zoom = 10

In [59]:
# define Enum class
class DrawState(Enum):
    IDLE=0
    PAINTING=1
    ERASING=2

class ShiftState(Enum):
    NOT_SHIFTING=0
    SHIFTING=1

#define a canvas class
#Canvas init, canvas to hold a pannable image of the map
class PannableCanvas(tk.Frame):
    
    #define parameters and initialize
    def __init__(self, master, image_path, max_width, max_height, current_hour, data_path, *args, **kwargs):

        #init parent class holding canvas
        super().__init__(master,*args,**kwargs)

        #create class args to be made public
        self.image_path = image_path
        self.image_original_width = None
        self.image_original_height = None
        self.data_path = data_path
        self.max_width = max_width
        self.max_height = max_height
        self.current_scale = 1.0
        self.tk_image = None
        self.roomdata_path = None # set in display_selected()
        self.rooms = pd.DataFrame(columns = ['id', 'name', 'coords']) # save state
        self.data = None
        self.memorystates = []
        self.current_hour = current_hour
        self.display_height = max_height
        self.display_width = max_width

        # shapes are the currently drawn shapes.
        # vertices are the currently in-progress drawn verts of the next shape before they get saved as a shape.
        
        self.shapes = [] # for drawing purposes
        self.lines = [] # for drawing purposes
        
        self.vertices = np.empty(shape=(0, 2)) # shape=(n_verts, 2) in json coordinate space
        self.mouseline = None
        self.draw_state = DrawState.IDLE
        self.shift_state = ShiftState.NOT_SHIFTING
        self.orientation = 1
        
        #canvas init
        self.canvas = tk.Canvas(self, width=max_width, height=max_height, bg='white')
        self.canvas.grid(row=0,column=0, sticky = "nsew")
        
        self.grid_rowconfigure(0, weight=1)
        self.grid_columnconfigure(0, weight=1)
        
        #image init
        self.image = self.scale_image(image_path)

        self.canvas_image = self.canvas.create_image(0,0, anchor=tk.NW, image=self.image)
        self.canvas.config(scrollregion=self.canvas.bbox(tk.ALL))
        
        
        #mouse init
        self.key_rebind()

        #center canvas
        if self.image != None:
            self.center_canvas()

    #a function wrapper to run 2 functions at once for event binding
    def on_mouse_motion(self, event):
        self.hover_over(event)
        self.draw_mouseline(event)

    #using the roomdata path, find current existing csv or create csv of rooms of that floor
    def extract_rooms(self):
        print(f"extracting rooms from {self.roomdata_path}")
        df = pd.DataFrame(columns = ['id', 'name', 'coords'])
        try:
            maybe_valid = pd.read_csv(self.roomdata_path)
            if list(df.columns) != ['id', 'name', 'coords']:
                print("Invalid format. Not reading the file (it will get wiped very soon).")
            else:
                # looks good
                print("Found valid roomdata_path file (.csv file)")
                maybe_valid['coords'] = maybe_valid['coords'].apply(lambda x: np.array(json.loads(x.replace(' ', ','))))
                df = maybe_valid
                

        except:
            print("File not found in roomdata_path.")
                    
        self.rooms = df
        self.update_image()

    #extract the data of how many people are in a room
    def extract_data(self):
        try:
            df = pd.read_csv(self.data_path)
        except:
            tk.messagebox.showwarning(title="File not Found", message="Invalid path to file, file not found.")
            return

        #'people' column will be a list, with each index representing an hour 0 = 0:00,  23 = 23:00
        if list(df.columns) != ['id', 'people']:
            tk.messagebox.showwarning(title="Invalid Format", message="File format is invalid, file not used.")
            return

        self.update_image()
    
    #rebind keys when paint mode is on/off
    def key_rebind(self):
        #include shift for later when grid snapping is worked on
        if self.draw_state == DrawState.PAINTING:
            #code for paint mode
            self.canvas.unbind("<ButtonPress-1>")
            self.canvas.unbind("<B1-Motion>")
            self.canvas.unbind('<ButtonPress-3>')
            self.canvas.unbind('<KeyRelease>')
            self.canvas.unbind('<Control-z>')
            self.canvas.unbind('<Control-y>')
            self.canvas.bind("<ButtonPress-1>", self.start_pan)
            self.canvas.bind("<B1-Motion>", self.pan)
            self.canvas.bind("<ButtonPress-3>", self.create_vertex)
            self.canvas.bind("<Motion>", self.on_mouse_motion)
            self.bind("<KeyRelease>", self.can_create)
            self.canvas.bind("<KeyPress>", self.shifting)
            # self.canvas.bind("<Control-z>", self.undo)
            # self.canvas.bind("<Control-y>", self.redo)
        elif self.draw_state == DrawState.ERASING:
            self.canvas.unbind("<ButtonPress-1>")
            self.canvas.unbind("<B1-Motion>")
            self.canvas.unbind('<ButtonPress-3>')
            self.canvas.unbind('<KeyRelease>')
            self.canvas.unbind('<Control-z>')
            self.canvas.unbind('<Control-y>')
            self.canvas.unbind("<Motion>")
            self.canvas.bind("<Motion>", self.hover_over)
            self.canvas.bind("<ButtonPress-3>", self.delete_shape)
            self.canvas.bind("<ButtonPress-1>", self.start_pan)
            self.canvas.bind("<B1-Motion>", self.pan)
        elif self.draw_state == DrawState.IDLE:
            #code for not paint mode
            self.canvas.unbind("<ButtonPress-1>")
            self.canvas.unbind("<B1-Motion>")
            self.canvas.unbind('<ButtonPress-3>')
            self.canvas.unbind('<KeyRelease>')
            self.canvas.unbind('<Control-z>')
            self.canvas.unbind('<Control-y>')
            self.canvas.unbind("<Motion>")
            self.canvas.bind("<Motion>", self.hover_over)
            self.canvas.bind("<ButtonPress-1>", self.start_pan)
            self.canvas.bind("<B1-Motion>", self.pan)
        else:
            # this will be reached only if there's a bug
            raise Exception(f"Unhandled self.draw_state {self.draw_state}")
            
    # scale the image to the current scale
    # TODO initially set self.current_scale such that the image will fit within the canvas nicely.
    def scale_image(self, image_path):
        image_path = image_path.strip().strip("'").strip('"')
        try:
            image = Image.open(image_path)
        except Exception as e:
            print(f"Error opening image: {e}")
            return None
            
        self.image_original_width, self.image_original_height = image.size

        scaled_size = (int(self.image_original_width * self.current_scale), int(self.image_original_height * self.current_scale))
        
        return ImageTk.PhotoImage(image.resize(scaled_size))

    # redraw image and all shapes for when zoomin/zoomout is called or hour change
    def update_image(self):
        self.image = self.scale_image(self.image_path)
        if self.image:
            self.canvas.itemconfig(self.canvas_image, image=self.image)
            self.canvas.config(scrollregion=self.canvas.bbox(tk.ALL))
        else:
            print("No image loaded")
        
        self.draw_shapes()
        
        self.draw_vertices()

    #zoom in
    def zoom_in(self):
        if self.current_scale * 1.2 <= max_zoom:
            self.current_scale *= 1.2 
            self.update_image()
        print(f'current zoom: {self.current_scale}')
    #zoom out
    def zoom_out(self):
        if self.current_scale / 1.2 >= min_zoom:
            self.current_scale /= 1.2
            self.update_image()
        print(f'current zoom: {self.current_scale}')
            
    #track starting coords
    def start_pan(self, event):
        self.canvas.scan_mark(event.x, event.y)
        print(f'canvas-space coords: {self.convert_coords_window_to_canvas((event.x, event.y))}')


    # COORDINATE SPACES
    # window space: (0, 0) is top left of the screen, (screen_w,screen_h) is bottom right
    # canvas: (0, 0) is the top left of the canvas, (image_w,image_h) is bottom right of canvas
    # json: (0, 0) is the top left of the canvas, (1,1) is the bottom right of canvas
    
    def convert_coords_window_to_canvas(self, coords):
        x, y = coords
        return self.canvas.canvasx(x), self.canvas.canvasy(y)

    def convert_coords_canvas_to_json(self, coords):
        x, y = coords
        return x/self.image_original_width/self.current_scale, y/self.image_original_height/self.current_scale

    def convert_coords_json_to_canvas(self, coords):
        x, y = coords
        return x*self.image_original_width*self.current_scale, y*self.image_original_height*self.current_scale

    # shortcuts from window straight to json and back
    def convert_coords_window_to_json(self, coords):
        return self.convert_coords_canvas_to_json(self.convert_coords_window_to_canvas(coords))

    #drag with scan_dragto
    def pan(self, event):
        self.canvas.scan_dragto(event.x, event.y, gain=1)

    def snap_to_nearest_vertex(self, coords):
        x, y = coords
        np_vertices = np.array(self.vertices)

        if len(self.rooms['coords']) > 0:
            np_savedvertices = np.concatenate(self.rooms['coords'])
        else:
            np_savedvertices = np.array([])
        
        # acutally distance squared for performance but whateva
        distance_to_point = {}
        
        for vx, vy in np.append(np_vertices, np_savedvertices).reshape(-1,2):
            dx = vx - x 
            dy = vy - y
            dist_squared = dx**2 + dy**2
            if dist_squared < (snap_radius)**2:
                distance_to_point[vx, vy] = dist_squared
        
        return min(distance_to_point, key=distance_to_point.get, default=coords)
    
    
    def create_vertex(self, event):

        json_xy = self.convert_coords_window_to_json((event.x, event.y))

        #auto snap, check if current x,y is close to any x,y in the list
        if self.shift_state == ShiftState.NOT_SHIFTING:
             snapped_xy = self.snap_to_nearest_vertex(json_xy)


        
        # vstack will just append the snapped vertices to the vertices np.array by creating a new row.
        self.vertices = np.vstack((self.vertices, snapped_xy))

        print(f"creating vertex at {snapped_xy} new num vertices {self.vertices.size}")
        
        self.update_image()

    def draw_vertices(self):

        # erase all the existing lines
        # TODO maybe instead draw once and just update the one which moves with the cursor.
        for line in self.lines:
            self.canvas.delete(line)
        self.lines.clear()
            
        # scale the new coordiantes based off the scale
        canvas_vertex_coords = [self.convert_coords_json_to_canvas(coords) for coords in self.vertices]

        # define this lambda since its only used in this one function.
        # returns [] if len(
        sliding_window = lambda data, window_size, step_size: [data[i:i + window_size] for i in range(0, len(data) - window_size + 1, step_size)]

        #connect vertices from one to another in the list, from the bottom to top
        for start_vertex, end_vertex in sliding_window(canvas_vertex_coords, 2, 1):
            start_x, start_y = start_vertex
            end_x, end_y = end_vertex

            # draw the line
            # save it to a list so it can be erased later
            line = self.canvas.create_line(start_x, start_y, end_x, end_y, fill='black', width=4)
            self.lines.append(line)

    #while actively drawing a box, connect the last drawn vertex to mouse
    def draw_mouseline(self, event):
        if self.vertices.size == 0:
            return
            
        start_xy = self.convert_coords_json_to_canvas(self.vertices[-1])
        end_xy = self.convert_coords_window_to_json((event.x, event.y))

        if self.shift_state == ShiftState.NOT_SHIFTING:
            end_xy = self.snap_to_nearest_vertex(end_xy)

        end_xy = self.convert_coords_json_to_canvas(end_xy)
        
        #clear previous line
        self.canvas.delete(self.mouseline)

        #draw new line
        start_x, start_y = start_xy
        end_x, end_y = end_xy
        self.mouseline = self.canvas.create_line(start_x, start_y, end_x, end_y, fill="black", width=2)

    # raycast algorithm to detect whether mouse is in a shape
    # param xy_json is in json coordinate space.
    # returns None if no intersections or the INDEX in self.rooms of the clicked-on room 
    def raycast_detect(self, xy_json):
        
        point = Point(1, 1)
        
        for index, coords in self.rooms['coords'].items():

            polygon = Polygon(coords.reshape(-1, 2))

            if polygon.contains(point):
                return index

        return None
                
            
    def delete_shape(self, event):
        
        xy_json = self.convert_coords_window_to_json((event.x, event.y))
        active_room = self.raycast_detect(clicked_point)
        
        #delete elements of selected room
        if active_room != None:
            self.rooms.drop(active_room, inplace=True)
            self.update_image()
    
    #detect whether the mouse is hovered over a shape using ray casting, with hover, display room id and # of ppl, in delete mode, right clicking will delete
    def hover_over(self, event):
        if self.image:
            global label
    
            xy_json = self.convert_coords_window_to_json((event.x, event.y))
            active_room = self.raycast_detect(xy_json)
            
            #Add number of people later
            if active_room != None:
                txt_roomname = self.rooms['name'].loc[active_room]
                txt_roomid = self.rooms['id'].loc[active_room]
                
                txt_roomppl = "not implemented"

                label.pack_forget()
                label = tk.Label(top_left, text=f"Room {txt_roomid}: {txt_roomname} Occupancy: {txt_roomppl}", width= 40, height=4, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground='grey', font=("Arial", 12))
                label.pack(side='left', padx=50, pady=5)
            else:
                label.pack_forget()
                label = tk.Label(top_left, text="Room <id>: <name> Occupancy: <> ", width= 40, height=4, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground='grey', font=("Arial", 12))
                label.pack(side='left', padx=50, pady=5)

    
    #check if shape can be created based off key-release
    def can_create(self, event):
        if event.keysym == 'Return':    
            # if len(self.vertices) > 4:
            self.create_shape()

        #also detect if the keyrelease is left shift to make is_shifting false, as this is a keyrelease detect funtion
        if event.keysym == 'Shift_L':
            self.shift_state = ShiftState.NOT_SHIFTING
        
    def shifting(self, event):
        if event.keysym == 'Shift_L':
            self.shift_state = ShiftState.SHIFTING

    # prompt is the text to display to the user when asking for user input
    # condition is a lambda String: Boolean. input is the user provided string and 
    # output is true if the input is good and false if the function should continue
    # asking for user input.
    # message - the message to show if the output of parameter condition is false.
    # returns the user's input or None if user cancels.
    def ask_for_user_input_until_valid(self, prompt, condition=lambda x: True, on_fail_message="Debug: Custom condition failed."):
        user_input = None
        while user_input is None:
            user_input = simpledialog.askstring("Input", prompt)
            if user_input is None:  # The user canceled the input
                messagebox.showwarning("Warning", "Operation cancelled")
                print("user input cancelled")
                return None
            elif not user_input.strip():  # The user entered an empty string
                messagebox.showwarning("Warning", "Input cannot be blank!")
                print("user input was blank")
                user_input = None
            elif not condition(user_input):
                messagebox.showwarning("Warning", on_fail_message)
                print("user input custom fail")
                user_input = None

        return user_input
    
    
    #take current placed vertices and create a shape that can be pushed into csv and pandas df for other usages
    def create_shape(self):

        # ask for room id. Additionally, if it already exists in the rooms, ask again.
        room_id = self.ask_for_user_input_until_valid(
            "Enter room id:",
            lambda x: x not in self.rooms['id'].astype(str).values, # must convert the rooms[id] to string in case they are all ints.
            "Already existing input, please enter a different id"
        )

        # user cancelled the thing.
        if room_id == None:
            # delete vertices memory
            self.vertices = np.empty(shape=(0, 2))
            
            # erase all lines from UI
            for line in self.lines:
                self.canvas.delete(line)
                
            # erase lines memory
            self.lines.clear()
            return

        # ask for room name. Duplicates allowed so no need to check the dataframe.
        room_name = self.ask_for_user_input_until_valid("Enter room name:")

        # user cancelled the 
        if room_name == None:
            return
 
        self.rooms = pd.concat([self.rooms, pd.DataFrame({
                                    'id': [room_id], 
                                    'name': [room_name], 
                                    'coords': [self.vertices.flatten()]})])

        self.rooms.reset_index(drop=True, inplace=True)
    
        self.vertices = np.empty(shape=(0, 2))
        for line in self.lines:
            self.canvas.delete(line)
        self.lines.clear()
        
        self.canvas.delete(self.mouseline)
        self.update_image()

        # autosave!!!
        self.save_file()

        #calculate colour of shape from green to red based off of how many people
    def calc_color(self, id):
        #scale of 1-10, 10 being max power
        if self.rooms.empty or self.rooms is None:
            room = self.data[self.data['id'] == id]
        else:
            return None
        try:
            n = room['people'][self.current_hour]
        except:
            messagebox.showwarning("Invalid Format", "Data could not be extracted properly, invalid data or invalid format")
            return None
            
        val = min(n/10, 1)
        red = int(val * 255)
        green = int((1-val) * 255)

        #translate into hex
        color = f'#{red:02x}{green:02x}00'
        return color

    
    #draw all rooms of the current hour to be overlayed on the image using coordinates and draw.polygon()
    #going to need another parameter, "people" to take in a pandas object to associate # of ppl per room
    def draw_shapes(self):

        # erase old shapes
        for shape in self.shapes:
            self.canvas.delete(shape['id'])

        self.shapes.clear()

        if self.rooms is None or self.rooms.empty:
            return

        # draw new shapes
        for _, room in self.rooms.iterrows():
            # #init values for polygon
            color = self.calc_color(room['id'])
            alpha = 128
            coords = [self.convert_coords_json_to_canvas(coords) for coords in room['coords'].reshape(-1, 2)]
            
            #create image with RGBA values for translucent effect
            img = Image.new('RGBA', (self.image_original_width, self.image_original_height), (0, 0, 0, 0))
            draw = ImageDraw.Draw(img)
    
            #fill the image with a polygon of calculated colour
            #polygons by default do not take alpha values, hence the image was created
            if color != None:
                draw.polygon(coords, fill=(int(color[1:3], 16), int(color[3:5], 16), int(color[5:7], 16), alpha), outline='black')
            else:
                draw.polygon(coords, fill=(255,0,0,128), outline='black')
                
            tk_img = ImageTk.PhotoImage(img)
    
            image_item = self.canvas.create_image(0, 0, anchor=tk.NW, image=tk_img)
    
            #add shapes into list for easy access when updating
            self.shapes.append({'id': image_item, 'original_coords': coords, 'image': tk_img})

    #when finished with paint mode, save the pandas dataframe into a csv file in roomdata_path
    def save_file(self):
        self.rooms.to_csv(self.roomdata_path, index=False)
    
    #center the canvas
    def center_canvas(self):
        canvas_width = self.canvas.winfo_width()
        canvas_height = self.canvas.winfo_height()
        image_width = self.image.width()
        image_height = self.image.height()

        if image_width > canvas_width:
            self.canvas.xview_moveto((image_width - canvas_width) / 2 / image_width)
        else:
            self.canvas.xview_moveto(0.5 - canvas_width / image_width / 2)

        if image_height > canvas_height:
            self.canvas.yview_moveto((image_height - canvas_height) / 2 / image_height)
        else:
            self.canvas.yview_moveto(0.5 - canvas_height / image_height / 2)

        # Adjust the scroll region to accommodate zoom
        self.canvas.config(scrollregion=self.canvas.bbox(tk.ALL))


In [60]:
#helper functions (they operate on the global objects)
def zoom_out():
    canvas.zoom_out()

def zoom_in():
    canvas.zoom_in()

#update value of the slider to show the data of specified hour
def slider_change(value):
    canvas.current_hour = int(value)
    canvas.update_image()

#function for changing the hour for button_change()
def change_hour():
    global after_id
    current_val = slider.get()
    if current_val == 23:
        current_val = 0
    if button_video.image == pause_img:
        if current_val + 1 < 24:
            current_val += 1
            slider.set(current_val)
            canvas.update_image()
            after_id = root.after(1000, change_hour)
        else:
            return
  
#play pause function for play pause button
def button_change():
    global after_id
    #if the images are currently paused
    if button_video.image == play_img:
        button_video.config(image = pause_img)
        button_video.image = pause_img
        change_hour()
    else:
        #if the iamges are currently playing
        button_video.config(image = play_img)
        button_video.image = play_img
        if after_id is not None:
            root.after_cancel(after_id)  # Cancel the scheduled increment
            after_id = None

#rotating the image for the rotate button
def button_rotate():
    #delete later, currently cannot solve rotation for the life of me
    if canvas.image:
        canvas.orientation += 1
        if canvas.orientation > 4:
            canvas.orientation = 1
        canvas.update_image()

#update the listbox
def update_listbox(data):
    #first clear the listbox
    listbox.delete(0, tk.END)

    #upload new data
    for item in data:
        listbox.insert(tk.END, item)


#button function to upload directoy
def upload_dir():
    dirname_data = filedialog.askdirectory(title="Select a directory", initialdir=default_search_path)
    if not dirname_data:
        return

    #walk through all sub directories and get to files
    for root, dirs, files in os.walk(dirname_data):
        for file in files:
            file_path = os.path.join(root, file)
            if file.lower().endswith('pdf'):
                #Potential issue here with fitz.open not using the full absolute path, instead it is only using the name
                pdf = fitz.open(file)
                for i in range(len(pdf)): 
                #save output of jpg
                    page = pdf.load_page(i)
                    img = page.get_pixmap()
                    newpdf_name = f'page{i+1}_{name}.png'
                    newpdf_path = os.path.join(temp_dir, newpdf_name)
                    img.save(newpdf_path)
                    filename_database.add(newpdf_path)
            elif file.lower().endswith('png'):
                 filename_database.add(file_path)

    #update listbox
    update_listbox(filename_database)

#requests a file destination input from user, stores path into database
def upload_image():
    filename_data = filedialog.askopenfilename(title="Select a file", filetypes=(("PNG", "*.png"), ("PDF", "*.pdf")))
    if not filename_data:
        return
    
    #check if pdf
    _, extension = os.path.splitext(filename_data)
    
    if extension == ".pdf":
        #convert from pdf into jpg
        pdf = fitz.open(filename_data)
        for i in range(len(pdf)): 
            #save output of jpg
            page = pdf.load_page(i)
            img = page.get_pixmap()
            newpdf_name = f'page{i+1}_{name}.png'
            newpdf_path = os.path.join(temp_dir, newpdf_name)
            img.save(newpdf_path)
            filename_database.add(newpdf_path)

    else:
        filename_database.add(filename_data)

    update_listbox(filename_database)

#update the listbox based on what's currently inside the search box
def search_filter(event):
    value = searchbox.get()

    if value == '':
        data = filename_database
    else:
        data = []

        #loop through all items of the database and add what has value in it
        for item in filename_database:
            if value.lower() in item.lower():
                data.append(item)

    #send the new list to listbox
    update_listbox(data)

#uplaod the csv data to assign id numbers to how many people
def upload_data():
    filename_data = filedialog.askopenfilename(title="Select a file", filetypes=(("CSV", "*.csv"),))
    canvas.data_path = filename_data
    canvas.extract_data()

#grab selection in combobox, and then display on canvas
def display_selected(event):
    global listbox
    global canvas
    #grab the filename from combobox
    option_name = listbox.get(listbox.curselection())
    #using filename as a key, get filename's path
    #set canvas roomdata path to represent the combobox selection
    canvas.roomdata_path = os.path.join(roomdata_path, os.path.splitext(option_name)[0]+'.csv')
    canvas.extract_rooms()
    canvas.image_path = option_name
    canvas.update_image()
    canvas.center_canvas()
    canvas.pack(fill=tk.BOTH, expand=True)

def set_state_to_idle_and_update_ui():
    canvas.draw_state = DrawState.IDLE
    update_ui_for_state()
    canvas.key_rebind()

def set_state_to_erasing_and_update_ui():
    canvas.draw_state = DrawState.ERASING
    update_ui_for_state()
    canvas.key_rebind()

def set_state_to_painting_and_update_ui():
    canvas.draw_state = DrawState.PAINTING
    update_ui_for_state()
    canvas.key_rebind()

# enable painting mode and its features
def update_ui_for_state():
    global paint_button
    global save_button
    
    if listbox.get(tk.ACTIVE) == '':
        tk.messagebox.showwarning(title="Missing Image", message="Missing image, please upload an image first.")
    else:
        if canvas.draw_state == DrawState.IDLE:
            #button pack and unpack
            save_button.pack_forget()
            paint_button = tk.Button(right_frame, text = "Paint", width= 10, height = 2, relief='solid',bg='white', activebackground='white', activeforeground='black',  borderwidth=1, highlightbackground = 'grey', command=set_state_to_painting_and_update_ui)
            paint_button.pack(side = 'bottom', pady = 10)
        
        elif canvas.draw_state == DrawState.PAINTING:
            canvas.focus_set()

            #button pack and unpack
            paint_button.pack_forget()
            #code erase mode for erase_button later
            save_button = tk.Button(right_frame, text= "Normal", width=10, height=2, relief="solid", bg="white", activebackground='white', activeforeground='black', borderwidth=1, highlightbackground='grey', command=set_state_to_idle_and_update_ui)
            save_button.pack(padx=5, pady=5)
            paint_button = tk.Button(right_frame, text = "Erase", width=10, height = 2, relief="solid", bg="white", activebackground='white', activeforeground='black', borderwidth=1, highlightbackground='grey', command=set_state_to_erasing_and_update_ui)
            paint_button.pack(padx=5, pady=5)
        
        elif canvas.draw_state == DrawState.ERASING:
            #button pack and unpack
            paint_button.pack_forget()
            #code erase mode for erase_button later
            save_button = tk.Button(right_frame, text= "Normal", width=10, height=2, relief="solid", bg="white", activebackground='white', activeforeground='black', borderwidth=1, highlightbackground='grey', command=set_state_to_idle_and_update_ui)
            save_button.pack(padx=5, pady=5)
            paint_button = tk.Button(right_frame, text = "Paint", width=10, height = 2, relief="solid", bg="white", activebackground='white', activeforeground='black', borderwidth=1, highlightbackground='grey', command=set_state_to_painting_and_update_ui)
            paint_button.pack(padx=5, pady=5)


def defocus(event):
    event.widget.master.focus_set()

#window close protocols
def on_close():
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
        root.destroy()


In [61]:
#define a bunch of global ui elements

#frame and button init
top_frame = tk.Frame(root, bg = 'white')
top_frame.pack(side = tk.TOP)
top_left = tk.Frame(top_frame, bg='white')
top_left.pack(side = "left")

# rotate_path = Image.open(r'rotate-left.png')
# rotate_path = rotate_path.resize((30,30), Image.LANCZOS)
# rotate_img = ImageTk.PhotoImage(rotate_path)
# button_rotate = tk.Button(top_frame, image=rotate_img,  width=35, height=35, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground = 'grey', command=button_rotate)
# button_rotate = tk.Button(top_frame,  text="rotate", width=35, height=35, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground = 'grey', command=button_rotate)
# button_rotate.pack(side = 'right', padx=10, pady=10)

button_zoomin = tk.Button(top_frame, text = "+", width=4, height=2, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground = 'grey', command=zoom_in)
button_zoomin.pack(side = 'right', expand = False, pady=10)
button_zoomout = tk.Button(top_frame, text = "-", width=4, height=2, relief='solid',bg='white', activebackground='white', activeforeground='black',  borderwidth=1, highlightbackground = 'grey', command=zoom_out)
button_zoomout.pack(side = 'right', expand = False, pady=10)

right_frame = tk.Frame(root, bg = 'white')
right_frame.pack(side = tk.RIGHT, padx = 10)
right_bottom = tk.Frame(right_frame, bg = 'white')
right_bottom.pack(side="bottom")

#Searchbox
searchbox = tk.Entry(right_frame, font=("Arial", 12), background = '#DDDDDD')
searchbox.pack(side= 'top', padx=10)
searchbox.bind('<KeyRelease>', search_filter)

#Listbox for searched values
listbox = tk.Listbox(right_frame, width = 30)
listbox.pack(side = 'top', padx=10, pady=10, expand= True)
listbox.bind('<<ListboxSelect>>', display_selected)
#combobox for map selection
# combobox = ttk.Combobox(right_frame)
# combobox.bind("<<ComboboxSelected>>", display_selected)
# combobox.pack(side = 'top', padx=10)

#scrollable listbox 
# scrollbox = create_scrollbox(right_frame)


#Paint mode button
paint_button = tk.Button(right_bottom, text = "Paint", width= 10, height = 2, relief='solid',bg='white', activebackground='white', activeforeground='black',  borderwidth=1, highlightbackground = 'grey', command=set_state_to_painting_and_update_ui)
paint_button.pack(side = 'bottom', pady = 10)

#label for captions 
label = tk.Label(top_left, text="Room Id:  # of People: ", width= 40, height=4, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground='grey', font=("Arial", 12))
label.pack(side='left', padx=10, pady=5)

#load images for play/pause button
play_path = Image.open(r'play.png')
play_path = play_path.resize((35,35), Image.LANCZOS)
pause_path = Image.open(r'pause.png')
pause_path = pause_path.resize((35,35), Image.LANCZOS)
play_img = ImageTk.PhotoImage(play_path)
pause_img = ImageTk.PhotoImage(pause_path)
# button_video = tk.Button(top_frame, image=play_img, width=35, height=35, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground = 'grey', command=button_change)
button_video = tk.Button(top_frame, text="video", width=35, height=35, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground = 'grey', command=button_change)
button_video.image = play_img
button_video.pack(side = 'left', pady=10)

#upload dir with images
uploaddir_button = tk.Button(top_frame, text="Upload Directory", width=13, height=2, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground = 'grey', command=upload_dir)
uploaddir_button.pack(side = "left", pady=10, padx =10)

#Upload image button
uploadimg_button = tk.Button(top_frame, text="Upload Image", width=13, height=2, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground = 'grey', command=upload_image)
uploadimg_button.pack(side = "left", pady=10, padx =10)

#upload room ppl data
uploaddata_button = tk.Button(top_frame, text="Upload Data", width=13, height=2, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground = 'grey', command=upload_data)
uploaddata_button.pack(side= 'left', pady=10)

slider = tk.Scale(top_frame, from_ = 0, to=23, orient='horizontal', command=slider_change, length = 300)
slider.pack(pady=20, padx = 20)

hour = 0

#canvas init
canvas = PannableCanvas(root, image_path, max_width, max_height, hour, data_path)
canvas.pack(fill=tk.BOTH, expand=True)

slider.config(resolution= 1)

In [62]:
#window close protocols
root.protocol("WM_DELETE_WINDOW", on_close)

# Run the main event loop
root.mainloop()

extracting rooms from /Users/stefan/Documents/life/work/network/localization/app/wifiscans/floormaps/Administration/2nd Floor.csv
Found valid roomdata_path file (.csv file)
canvas-space coords: (1534.0, 541.0)
canvas-space coords: (1249.0, 508.0)
current zoom: 0.8333333333333334
current zoom: 0.6944444444444445
current zoom: 0.5787037037037038
current zoom: 0.48225308641975323
current zoom: 0.401877572016461
canvas-space coords: (1046.0, 323.0)


In [63]:
# from tkinter import *

# def fun(event):
#     print(event.keysym, event.keysym=='a')
#     print(event)

# root = Tk()

# root.bind("<KeyRelease>", fun)
# root.mainloop()

In [64]:
# import ast
# df = pd.read_csv('CCIS2-NLT/L1.csv')
# df['coords'] = df['coords'].apply(ast.literal_eval)
# df.rename(columns={'coords': 'polygon'}, inplace=True)
# df.to_dict(orient='records')

In [65]:
arr = np.empty((0,2))
arr = np.vstack((arr, (0.0, 0.0)))
arr = np.vstack((arr, (2.0, 0.0)))
arr = np.vstack((arr, (2.0, 2.0)))
arr = np.vstack((arr, (0.0, 2.0)))
x = [canvas.convert_coords_json_to_canvas(coords) for coords in arr]

sliding_window = lambda data, window_size, step_size: [data[i:i + window_size] for i in range(0, len(data) - window_size + 1, step_size)]

sliding_window(x, 2, 1)

[[(0.0, 0.0), (2485.2109053497948, 0.0)],
 [(2485.2109053497948, 0.0), (2485.2109053497948, 1041.666666666667)],
 [(2485.2109053497948, 1041.666666666667), (0.0, 1041.666666666667)]]